In [1]:
#download required dependencies 
!pip install geopandas
!pip install netCDF4

     |████████████████████████████████| 901kB 9.6MB/s 
     |████████████████████████████████| 17.9MB 42.3MB/s 
     |████████████████████████████████| 11.2MB 39.7MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=1add48b354cb1ce642e85b67711202a9133bce7e4718627bf662b3b191866442
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
     |████████████████████████████████| 4.1MB 9.6MB/s 
     |████████████████████████████████| 307kB 60.1MB/s 


In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#load in functions required and dependencies to use them
%run all_functions.ipynb

In [ ]:
#get flow + met data
flowDict = get_ProvinceDailyStreamflow(colab=1)
precDict = get_ProvinceDailyPrec('AB',colab=1)
tempDict = get_ProvinceDailyTemp('AB',colab=1)

In [ ]:
#unpack data

stationLat = flowDict['stationLat']
stationLon = flowDict['stationLon']
eraLat = tempDict['latERA']
eraLon = tempDict['lonERA']

flowDays = flowDict['windowDays']
flowMonths = flowDict['windowMonths']
flowYears = flowDict['windowYears']
eraDays = tempDict['daysERA']
eraMonths = tempDict['monthsERA']
eraYears = tempDict['yearsERA']

F = flowDict['all_flowwindow_norm_NF'] #normalized discharge with nans filled (NF)
T = tempDict['T']
P = precDict['P']

In [ ]:
#clip to make data have same time range
startYear = max(int(np.min(eraYears)),int(np.min(flowYears)))

indStartERA = min(np.argwhere(eraYears==startYear))[0]
indStartFlow = min(np.argwhere(flowYears==startYear))[0]

F = F[indStartFlow:]
T = T[indStartERA:]
P = P[indStartERA:]

flowDays = flowDays[indStartFlow:]
flowMonths = flowMonths[indStartFlow:]
flowYears = flowYears[indStartFlow:]

eraDays = eraDays[indStartERA:]
eraMonths = eraMonths[indStartERA:]
eraYears = eraYears[indStartERA:]

In [ ]:
#prep data: make training and testing sets

#indices of testing/training
trainStartYear = 1987
trainFinYear = 2000
testStartYear = 2001
testFinYear = 2010

trainInds = np.squeeze(np.argwhere((flowYears>=trainStartYear) & (flowYears<=trainFinYear)))
testInds = np.squeeze(np.argwhere((flowYears>=testStartYear) & (flowYears<=testFinYear)))

#standardize variables individually (normalize wrt training period), then save as 32-bit rather than 64-bit for space
Tmean_train = np.mean([T[trainInds[ii]] for ii in range(len(trainInds))])
Tstd_train = np.std([T[trainInds[ii]] for ii in range(len(trainInds))])
Tnorm = (T - Tmean_train)/Tstd_train
Tnorm = np.single(Tnorm)

Pmean_train = np.mean([P[trainInds[ii]] for ii in range(len(trainInds))])
Pstd_train = np.std([P[trainInds[ii]] for ii in range(len(trainInds))])
Pnorm = (P - Pmean_train)/Pstd_train
Pnorm = np.single(Pnorm)

Fmean_train = np.nanmean([F[ii][trainInds[366:]] for ii in range(len(F))])
Fstd_train = np.nanstd([F[ii][trainInds[366:]] for ii in range(len(F))])
Fnorm = (F - Fmean_train)/Fstd_train
Fnorm = np.single(Fnorm)

#target data
y_train = np.squeeze([Fnorm[ii][366:len(trainInds)+366] for ii in range(len(F))])
y_test = np.squeeze([Fnorm[ii][testInds[1:]] for ii in range(len(F))])

#first, make (n_time x n_lon x n_lat x n_vars) tensor 
#x_intermediate = np.zeros((8766,17,43,2))
x_intermediate = np.empty((8766,17,43,2),dtype='single')
x_intermediate[:,:,:,0] = Tnorm
x_intermediate[:,:,:,1] = Pnorm
x_train_intermediate = x_intermediate[trainInds]
x_test_intermediate = x_intermediate[testInds]


In [ ]:
[np.shape(x_test), np.shape(y_test), np.shape(x_train), np.shape(y_train)]

In [ ]:
#now, convert x_intermediate into (n_time x 365 x n_lon x n_lat x n_vars) tensor
x = np.empty((8766-365,365,17,43,2),dtype='single')
x_train = np.empty((len(trainInds),365,17,43,2),dtype='single')
x_test = np.empty((len(testInds)-1,365,17,43,2),dtype='single')

In [ ]:
for ii in range(1000):
    x_train[ii] = x_intermediate[ii:ii+365]

In [30]:
for ii in range(1000,2000):
    x_train[ii] = x_intermediate[ii:ii+365]

[1826, 1461, 364, 1825]

In [ ]:
for ii in range(2000,3000):
    x_train[ii] = x_intermediate[ii:ii+365]

In [ ]:
for ii in range(3000,4000):
    x_train[ii] = x_intermediate[ii:ii+365]

In [ ]:
for ii in range(4000,len(trainInds)):
    x_train[ii] = x_intermediate[ii:ii+365]

In [ ]:
for ii in range(1000):
    x_test[ii] = x_intermediate[ii+len(trainInds)-365:ii+len(trainInds)]

In [ ]:
for ii in range(1000,2000):
    x_test[ii] = x_intermediate[ii+len(trainInds)-365:ii+len(trainInds)]

In [ ]:
for ii in range(2000,len(testInds)-1):
    x_test[ii] = x_intermediate[ii+len(trainInds)-365:ii+len(trainInds)]

In [ ]:
saveTrainVars=0
if saveTrainVars:
    np.savez("T_P_train_1987_2000_era_interim", x_train)
    #np.savez_compressed("T_P_train_1987_2000_era_interim_compressed", x_train)
    
saveTestVars=0
if saveTestVars:
    np.savez("T_P_test_1987_2000_era_interim", x_test)
    #np.savez_compressed("T_P_test_1987_2000_era_interim_compressed", x_test)

In [33]:
import keras
from keras import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense

#CNN model
print('Building model...')
model = Sequential()
model.add(TimeDistributed(
    Conv2D(filters = 8, kernel_size = (3,3), activation='relu',data_format='channels_last'), 
    input_shape=(365,17,43,2)))
model.add(TimeDistributed(MaxPooling2D(pool_size = 2)))
model.add(TimeDistributed(Flatten()))

#LSTM model with time-distributed CNN as input
model.add(LSTM(units = 20, activation='tanh', recurrent_activation='hard_sigmoid', 
               use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
               bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
               recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
               kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, implementation=1, 
               return_sequences=False, return_state=False))
model.add(Dense(194, activation = 'tanh'))

#compile
print('Compiling model...')
model.compile(loss=keras.losses.MSE,
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=['mae'])

Using TensorFlow backend.
W0807 23:41:46.893978 140006467065728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 23:41:46.930340 140006467065728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 23:41:46.937351 140006467065728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 23:41:46.972561 140006467065728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Building model...


W0807 23:41:47.212259 140006467065728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Compiling model...


In [ ]:
model.summary()

In [34]:
#fit
batch_size = 64
epochs = 3
model.fit(x_train, y_train.T,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test.T))

ValueError: ignored

In [38]:
y_test

array([], dtype=float64)

In [ ]:
print('Calculating predicted training values...')
y_train_predicted = model.predict(x_train)

print('Calculating predicted testing values...')
y_test_predicted = model.predict(x_test)

In [ ]:
model.save('model_initial_test.h5')

In [ ]:
keras.models.load_model('model_initial_test.h5')